Submit here:
https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw03

## Question 1 (1 point): Dummies on Month and Week-of-Month

Find the ABSOLUTE CORRELATION VALUE of the most correlated dummy <month-week_of_month> with the binary outcome variable is_positive_growth_5d_future?

You saw in the correlation analysis and modeling that September and October may be important seasonal months. In this task, we'll go futher and try to generate dummies for Month and Week-of-month (starting from 1). For example, the first week of October should be coded similar to this: 'October_w1'. Once you've generated the new set of variables, find the most correlated (in absolute value) one with is_positive_growth_5d_future and round it to 3 digits after the comma.

NOTE: new dummies will be used as features in the next tasks, please leave them in the dataset.

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt
     

In [2]:
df_full = pd.read_parquet("stocks_df_combined_2024_05_07.parquet.brotli", )

In [3]:
df = df_full[df_full.Date>='2000-01-01']


In [4]:
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']

CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type']

TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]

TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV

In [7]:
df['ln_volume'] = df.Volume.apply(lambda x: np.log(x))

/tmp/ipykernel_44666/1818720232.py:1: RuntimeWarning: divide by zero encountered in log
  df['ln_volume'] = df.Volume.apply(lambda x: np.log(x))
/tmp/ipykernel_44666/1818720232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ln_volume'] = df.Volume.apply(lambda x: np.log(x))


In [8]:
 # manually defined features
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
GROWTH

CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']
     

TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']

TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
print(f'Technical patterns count = {len(TECHNICAL_PATTERNS)}, examples = {TECHNICAL_PATTERNS[0:5]}')

MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10']
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO

Technical patterns count = 61, examples = ['cdl2crows', 'cdl3blackrows', 'cdl3inside', 'cdl3linestrike', 'cdl3outside']


In [16]:
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

/tmp/ipykernel_44666/3087586600.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '1' '2' ... '4' '0' '1']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[:,'Weekday'] = df.Weekday.astype(str)


In [12]:
df['month_wom'] = df.apply(lambda row: f'{row["Month"]}-{(row["Date"].day-1)//7+1}', axis=1)

/tmp/ipykernel_44666/1038171603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month_wom'] = df.apply(lambda row: f'{row["Month"]}-{(row["Date"].day-1)//7+1}', axis=1)


In [13]:
df['month_wom']

3490    January-1
3491    January-1
3492    January-1
3493    January-1
3494    January-1
          ...    
5422      April-5
5423        May-1
5424        May-1
5425        May-1
5426        May-1
Name: month_wom, Length: 182675, dtype: object

In [14]:
CATEGORICAL.append('month_wom')

In [17]:
dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')


In [21]:
print(f'We have {len(dummy_variables.columns)} dummy vars')

We have 115 dummy vars


In [23]:
DUMMIES = dummy_variables.columns

In [22]:
df_with_dummies = pd.concat([df, dummy_variables], axis=1)

In [25]:
df_with_corr = df_with_dummies[DUMMIES].corrwith(df['is_positive_growth_5d_future'])

In [30]:
df_with_corr.apply(np.abs).sort_values(ascending=False)

month_wom_September-3    0.034537
month_wom_March-4        0.026058
month_wom_October-5      0.026023
month_wom_February-3     0.024578
month_wom_May-4          0.022264
                           ...   
Ticker_RELIANCE.NS       0.000106
Ticker_TTE               0.000063
month_wom_February-1     0.000059
month_wom_March-2        0.000036
month_wom_November-1     0.000033
Length: 115, dtype: float64